## Cleaning, Merging, and Initial Feature Engineering

In [1]:
# Import libraries 
import pandas as pd
import numpy as np

#### Step 1: Load in datasets

In [2]:
spray = pd.read_csv('assets/input/spray.csv')
weather = pd.read_csv('assets/input/weather.csv')
west_nile = pd.read_csv('assets/input/train.csv')
test = pd.read_csv('assets/input/test.csv')

#### Step 2: Clean weather dataset

In [3]:
# Investigate weather
weather.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [4]:
# Inspect number of rows and columns
weather.shape

(2944, 22)

The first step is to examine and handle missing data from weather. According to the data dictionary from NOAA, M represents missing data and T represents trace amounts in the applicable columns. 

In [5]:
# Identify number of missing rows per column
weather[weather=='M'].count()

Station           0
Date              0
Tmax              0
Tmin              0
Tavg             11
Depart         1472
DewPoint          0
WetBulb           4
Heat             11
Cool             11
Sunrise           0
Sunset            0
CodeSum           0
Depth          1472
Water1         2944
SnowFall       1472
PrecipTotal       2
StnPressure       4
SeaLevel          9
ResultSpeed       0
ResultDir         0
AvgSpeed          3
dtype: int64

In [6]:
# Water1 appears to have the same number of missing rows are total number of rows
weather['Water1'].unique()

array(['M'], dtype=object)

In [7]:
# Water1 column appears to only have missing values, so we'll drop it
weather = weather.drop('Water1', axis=1)

Next, we'll take a look at all the columns with a substantial number of M's (>1000), as indicated above. This includes Depart, Depth, and SnowFall. We will decide to either keep or drop these columns. 

In [8]:
# Summary of Depart, Depth, and SnowFall column values 
weather[['Depart', 'Depth', 'SnowFall']].describe()

,Depart,Depth,SnowFall
count,2944,2944,2944
unique,42,2,4
top,M,M,M
freq,1472,1472,1472


In [9]:
# Half of the depth data is either missing or 0
weather['Depth'].value_counts()

M    1472
0    1472
Name: Depth, dtype: int64

In [10]:
# 50% of the snowfall values are missing - likely one weather station does not record snowfall 
weather['SnowFall'].value_counts(normalize=True)['M']

0.5

In [11]:
# Drop depth and snowfall columns since they don't appear to have enough useful information
# Drop depart column, as we suspect that absolute temperatures are more important than relative temperatures 
weather.drop(['Depth', 'SnowFall', 'Depart'], axis=1, inplace = True)

Now that we have dropped columns with a significant amount of missing data, we will procede by only cleaning columns that we think we be useful for modeling. We are keeping in mind the theory that West Nile is more likely in hot and dry weather than cold and wet. 

In [12]:
weather.columns

Index(['Station', 'Date', 'Tmax', 'Tmin', 'Tavg', 'DewPoint', 'WetBulb',
       'Heat', 'Cool', 'Sunrise', 'Sunset', 'CodeSum', 'PrecipTotal',
       'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed'],
      dtype='object')

Since only Station 1 records sunrise and sunset information we will use station 1 values for station 2 on the appropriate day

In [13]:
# Station 2 does not record sunrise or sunset information so using the values from station 1
weather = weather.replace('-', np.nan)
weather['Sunset'].fillna(method = 'ffill', inplace = True)
weather['Sunrise'].fillna(method = 'ffill', inplace = True)

We will create a rain column based on the codesum column to indicate if it rained on a particular day

In [14]:
# Create Rain column based on CodeSum 
weather['Rain'] = weather['CodeSum'].apply(lambda x: 1 if 'RA' in x else 0)

In [15]:
# Define features by our perception of importance - want to quanitfy temp, humidity, wind, and precipitation
weather_features = ['Station', 'Date', 'Tavg', 'WetBulb', 'PrecipTotal',
                    'AvgSpeed', 'Heat', 'Cool', 'Tmax', 'Tmin', 'DewPoint', 'ResultSpeed', 'ResultDir', 
                   'StnPressure', 'SeaLevel', 'Rain', 'Sunrise', 'Sunset']

In [16]:
# Investigate good features for further cleaning
weather[weather_features].head()

,Station,Date,Tavg,WetBulb,PrecipTotal,AvgSpeed,Heat,Cool,Tmax,Tmin,DewPoint,ResultSpeed,ResultDir,StnPressure,SeaLevel,Rain,Sunrise,Sunset
0,1,2007-05-01,67,56,0.00,9.2,0,2,83,50,51,1.7,27,29.10,29.82,0,0448,1849
1,2,2007-05-01,68,57,0.00,9.6,0,3,84,52,51,2.7,25,29.18,29.82,0,0448,1849
2,1,2007-05-02,51,47,0.00,13.4,14,0,59,42,42,13.0,4,29.38,30.09,0,0447,1850
3,2,2007-05-02,52,47,0.00,13.4,13,0,60,43,42,13.3,2,29.44,30.08,0,0447,1850
4,1,2007-05-03,56,48,0.00,11.9,9,0,66,46,40,11.7,7,29.39,30.12,0,0446,1851


We need to strip spaces from our columns and impute values to 'M' and 'T'. Note,'T' only appears in PrecipTotal. For trace amounts in PrecipTotal, we assume a value of 0.005, which is below the minimum recorded positive amount.  There were only 2 observations of 'M' in PrecipTotal, we assume they are 0.

In [17]:
# Define dataframe that will be used as a mask to update weather later
# Remove spaces before T and M
stripped_weather = weather[weather_features].select_dtypes('object').apply(lambda x: x.str.strip())

In [18]:
# Replace PrecipTotal T and M values 
stripped_weather['PrecipTotal'].replace("T",'0.005',inplace=True)
stripped_weather['PrecipTotal'].replace("M", '0.00',inplace=True)

In [19]:
# Update weather dataframe with new values 
weather['PrecipTotal'] = stripped_weather['PrecipTotal']

In [20]:
# Identify remaining missing values in the features of interest
weather[weather_features][weather[weather_features] == 'M'].count()

Station         0
Date            0
Tavg           11
WetBulb         4
PrecipTotal     0
AvgSpeed        3
Heat           11
Cool           11
Tmax            0
Tmin            0
DewPoint        0
ResultSpeed     0
ResultDir       0
StnPressure     4
SeaLevel        9
Rain            0
Sunrise         0
Sunset          0
dtype: int64

Since there are so few missing values in AvgSpeed, WetBulb, Heat, Cool, StnPressure, and SeaLevel. We will drop those rows. We can infer the Tavg from averaging Tmin and Tmax.

In [21]:
# Identify row index and mean values to update Tavg
avgs = weather[['Tmax', 'Tmin']][weather['Tavg']=='M'].mean(axis=1)
weather.loc[avgs.index,'Tavg'] = avgs

In [22]:
# Drop missing rows from avgspeed,  wetbulb, heat, cool
mask = (weather['WetBulb'] == 'M') | (weather['AvgSpeed']=='M') | (weather['Heat'] == 'M') | (weather['Cool'] == 'M') | (weather['SeaLevel'] == 'M') | (weather['StnPressure'] == 'M')
weather.drop(weather.index[mask], inplace=True)

In [23]:
# Convert object dtypes into float
cols = ['Station','Tavg', 'WetBulb', 'PrecipTotal', 'AvgSpeed', 'Heat',
        'Cool', 'Tmin','Tmax', 'DewPoint', 'ResultSpeed', 
        'ResultDir', 'StnPressure', 'SeaLevel', 'Rain', 'Sunrise', 'Sunset']
weather[cols] = weather[cols].astype(float)

In [24]:
# Define longitude and latitude for the 2 weather stations
weather['latitude'] = weather['Station'].map(lambda x: 41.995 if x == 1.0 else 41.786)
weather['longitude'] = weather['Station'].map(lambda x: -87.933 if x==1.0 else -87.752)

#### Step 3: Cleaning train and test dataframes

In [25]:
# Inspect train dataset
west_nile.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


In [26]:
# Inspect train columns
west_nile.columns

Index(['Date', 'Address', 'Species', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy',
       'NumMosquitos', 'WnvPresent'],
      dtype='object')

We will clean features we think will be import for modeling only. Note that the test data does not contain NumMosquitos so we will also have to drop it from the train. 

In [27]:
# Picking promising columns to clean
west_nile_features = ['Date','Species','Longitude', 'Latitude', 'Trap']
target = ['WnvPresent']

In [28]:
# Check for NaNs in selected columns
west_nile[west_nile_features].isna().sum()

Date         0
Species      0
Longitude    0
Latitude     0
Trap         0
dtype: int64

In [29]:
# Check data types for train
west_nile[west_nile_features].dtypes

Date          object
Species       object
Longitude    float64
Latitude     float64
Trap          object
dtype: object

In [30]:
# Inspect test columns - NumMosquitos is missing 
test.columns

Index(['Id', 'Date', 'Address', 'Species', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy'],
      dtype='object')

In [31]:
# Check for missing test data 
test[west_nile_features].isna().sum()

Date         0
Species      0
Longitude    0
Latitude     0
Trap         0
dtype: int64

In [32]:
# Years included in test data
test.Date.map(lambda x: x[:4]).unique()

array(['2008', '2010', '2012', '2014'], dtype=object)

In [33]:
# Years included in train data
west_nile.Date.map(lambda x: x[:4]).unique()

array(['2007', '2009', '2011', '2013'], dtype=object)

Upon closer inspection, there are many duplicates in the data that only differ by the number of mosquitos. Since we are not modeling the number of mosquitos we will drop duplicates, keeping the record with the maximum WnvPresent score if there are inconsistencies. 

In [34]:
# Drop duplicate observations
west_nile_clean = west_nile.sort_values('WnvPresent', ascending = False).drop_duplicates(['Date', 'Trap', 'Species'], keep = 'first')

In [35]:
# Around 2000 observations were removed 
west_nile_clean.shape, west_nile.shape

((8475, 12), (10506, 12))

In [36]:
# Classes are still very unbalanced
west_nile_clean['WnvPresent'].value_counts(normalize = True)

0    0.946077
1    0.053923
Name: WnvPresent, dtype: float64

#### Step 4: Clean spray dataset

Spray data is available from 2011 and 2013. Our train data encapsules these 2 years, but the test data does not. Unfortunately it won't make sense to include this spray data in our model. We'll probably incorporate it into our cost-benefit analysis after model selection is complete in order to make recommendations about spray locations based on our predictions.

In [37]:
# Inspect spray head
spray.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [38]:
# Check for missing data
spray.isna().sum()

Date           0
Time         584
Latitude       0
Longitude      0
dtype: int64

In [39]:
# Years included in spray data
spray.Date.map(lambda x: x[:4]).unique()

array(['2011', '2013'], dtype=object)

#### Step 5: Merging Dataframes

In order to merge dataframes, we need to create a feature that will use the weather information from the closest weather station. First we define a function that calculates the distance between 2 points. Next we make a function that will determine the closest station to a given point based on distance. Then we will create a new feature that indicates the closest station to each trap. 

In [40]:
# Make a "nearest station" feature in west_nile dataframe in order to merge weather and west_nile

# Distance function between 2 points
def distance(lat1, long1, lat2, long2):
    return np.sqrt((lat1 - lat2)**2 + (long1-long2)**2)

# Closest station to each trap
def closest_station(lat, long):
    lat_1, long_1 = 41.995, -87.933
    lat_2, long_2 = 41.786, -87.752
    if distance(lat, long, lat_1, long_1) < distance(lat, long, lat_2, long_2):
        return 1.0
    else:
        return 2.0

In [41]:
# Create nearest station column in train and test dataframes
west_nile_clean['Station'] = west_nile_clean.apply(lambda x: closest_station(x['Latitude'], x['Longitude']),axis=1)
test['Station'] = test.apply(lambda x: closest_station(x['Latitude'], x['Longitude']),axis=1)

We're doing an inner join here for computational ease.  The most robust approach would impute missing weather values to observations in west_nile whose dates don't appear in the weather data. The weather data has dates that match all test dates, so no test data was dropped. 

In [42]:
# Merge dataframes in train and test
combined_train = pd.merge(west_nile_clean,weather,how='inner', on = ['Date', 'Station'])
combined_test = pd.merge(test, weather, how = 'inner', on = ['Date', 'Station'])

In [43]:
# Inspect merged dataframes
combined_train.columns

Index(['Date', 'Address', 'Species', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy',
       'NumMosquitos', 'WnvPresent', 'Station', 'Tmax', 'Tmin', 'Tavg',
       'DewPoint', 'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset', 'CodeSum',
       'PrecipTotal', 'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir',
       'AvgSpeed', 'Rain', 'latitude', 'longitude'],
      dtype='object')

#### Step 6: Save to CSV

In [44]:
# Create list of features to filter combined dataframes 
combined_features = west_nile_features[1:] + weather_features + target
combined_test_features = west_nile_features[1:] + weather_features

In [46]:
# Save train and test filtered dataframes to csv 
combined_train[combined_features].to_csv('assets/working/combined_train.csv',index=False)
combined_test[combined_test_features].to_csv('assets/working/combined_test.csv',index=False)

We will use the exported CSVs in our EDA and modeling going forward. Please refer to the EDA notebook for our next steps. 